In [1]:
import os
import re

from Bio import Entrez
from Bio import SeqIO

Entrez.email = "katerina.koptelova@yandex.ru"

## Определяем интересующие нас штаммы

Перечисляем названия организмов и штаммы, для которых хотим строить дерево (штамм можно не указывать)

In [2]:
# Археи
organisms = [ # (organism, strain)
    ('Archaeoglobus fulgidus', 'VC-16'),
    ('Ignicoccus hospitalis', 'KIN4/I'),
    ('Methanocaldococcus jannaschii', 'JCM 10045'),
    ('Methanococcus maripaludis', 'JJ'),
    ('Decoy', 'junk'),    # Такого в базах точно нет, скрипт отработает с предупреждением
    ('Pyrobaculum aerophilum', 'IM2'),
    ('Pyrobaculum calidifontis', 'VA1'),
    ('Pyrococcus horikoshii', 'JA-1'),
    ('Sulfolobus tokodaii', '7'),
]

# Бактерии
outgroup = [
    ('Desulfovibrio piger', 'ATCC 29098'),
    ('Herpetosiphon aurantiacus', ''),
    ('Rhodopirellula baltica', 'SH 1'),
]

## Получение данных из RefSeq

In [3]:
# 16S RefSeq Nucleotide sequence records
# https://www.ncbi.nlm.nih.gov/refseq/targetedloci/16S_process/
REFSEQ_16S_RNA_TERM = '33175[BioProject] OR 33317[BioProject]'    # Курируемая база 16S РНК бактерий и архей (очень хорошая)

# Задаем имена файлов
RAW_SEQ_FILENAME = '16S_sequences.fasta'    # файл с сырыми последовательностями
ALIGNEMENT_FILENAME = '16S_sequences_aligned.fasta'    # файл с выравниваниями
BLOCKS_FILENAME = '16S_sequences_blocks.phylip'    # файл с очищенными выравниваниями
ML_TREE_NAME = 'Bacteria_MLTree.newick'    # файл с деревом

In [4]:
full_names = {} # словарь с полными именами
taxonomy = {} # словарь с таксономией

with open(RAW_SEQ_FILENAME, 'w') as output_handle:    # открываем файл в режиме записи
    for organism, strain in organisms + outgroup:    # итерируемся по объединенному списку организмов (организмы + аутгруппа)
        
        # Запрос к базе данных
        query = f'"{organism}"[Organism]' # "название организма"[Strain]
        if strain:
            query += f' AND "{strain}"[Strain]'
        
        # Выполняем поиск
        # Я СКОПИПАСТИЛА ЭТО ИЗ ПРИМЕРА
        handle = Entrez.esearch(db='nucleotide',    # база данных поиска
                                term=f'({REFSEQ_16S_RNA_TERM}) AND ({query})',    # запрос к базе
                                sort='Date Modified')    # сортируем по свежести
        record = Entrez.read(handle)    # преобразуем handle в читаемый вид
        
        # Проверка на то, нашлось ли что-либо по запросу
        if int(record["Count"]) == 0:
            print(f'!!!\nWarning! No items found for {organism} {strain}\n')
            continue
        
        # Выбираем самый свежий результат и получаем последовательность
        seq_id = record['IdList'][0]    # берем самый свежий результат
        handle = Entrez.efetch(db='nucleotide',     # достаём (база данных поиска
                               id=seq_id,    # айдишник, получен строчкой выше
                               rettype="fasta")    # тип ответа)
        seq = SeqIO.read(handle, format='fasta')
               
        # Проверка на соответствие результата запросу
        if not (re.search(organism, seq.description) and re.search(strain, seq.description)):
            # если не (нашлось имя организма в результате и нашлось имя штамма в результате):
            print(f'!!!\nWarning! For "{organism} {strain}" was find just\n  {seq.description}\n')
            continue
        
        # Для верификации результата получим таксономические данные
        handle = Entrez.efetch(db='nucleotide', 
                               id=seq_id,
                               rettype="gb",
                               retmode='xml')
        taxonomy[seq.id] = Entrez.read(handle)[0]['GBSeq_taxonomy']    # запоминаем таксон по айдишнику чтобы в конце верифицировать дерево
        
        # В ходе обработки будем использовать короткие имена
        full_names[seq.id] = seq.description
        print(f'For        "{organism} {strain}" was retrieved:\n{seq.description}\n')
        seq.description = ''
        
        # Записываем последовательность в файл
        SeqIO.write(seq, output_handle, "fasta")

For        "Archaeoglobus fulgidus VC-16" was retrieved:
NR_118873.1 Archaeoglobus fulgidus DSM 4304 strain VC-16 16S ribosomal RNA, complete sequence

For        "Ignicoccus hospitalis KIN4/I" was retrieved:
NR_074104.1 Ignicoccus hospitalis strain KIN4/I 16S ribosomal RNA, complete sequence

For        "Methanocaldococcus jannaschii JCM 10045" was retrieved:
NR_113292.1 Methanocaldococcus jannaschii strain JCM 10045 16S ribosomal RNA, partial sequence

For        "Methanococcus maripaludis JJ" was retrieved:
NR_104984.1 Methanococcus maripaludis strain JJ 16S ribosomal RNA, partial sequence

!!!
Warning! No items found for Decoy junk

For        "Pyrobaculum aerophilum IM2" was retrieved:
NR_102764.2 Pyrobaculum aerophilum str. IM2 16S ribosomal RNA, complete sequence

For        "Pyrobaculum calidifontis VA1" was retrieved:
NR_040922.1 Pyrobaculum calidifontis JCM 11548 strain VA1 16S ribosomal RNA, complete sequence

For        "Pyrococcus horikoshii JA-1" was retrieved:
NR_029054.